libraries

In [13]:
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import cv2
%pip install data

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\utente\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [14]:
# Load the pre-trained face and smile cascade classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start the video capture (use webcam)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    gray = cv2.GaussianBlur(gray, (5, 5), 0.5)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 3,minSize=(100, 100))

    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Region of interest (ROI) for forehead detection
        x_forehead = x+int(w*0.1)
        w_forehead = int(w*0.8)
        y_forehead = y + int(h*0.05)
        h_forehead = int(h*0.2)
        forehead_region = gray[y_forehead:y_forehead + h_forehead, x_forehead:x_forehead + w_forehead] # Region around the forehead
        # Convert forehead ROI to grayscale and calculate average intensity
        intensity = np.mean(forehead_region)
        #show intensity
        cv2.putText(frame, f"Lum: {int(intensity)}", (x_forehead, y_forehead - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)


        # Draw rectangle around the forehead
        cv2.rectangle(frame, (x_forehead, y_forehead), (x_forehead+w_forehead, y_forehead + h_forehead), (0, 0, 255), 2)

        
    # Display the resulting frame
    cv2.imshow("Forehead frame", frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Definition for temperature 

In [15]:
def get_face_temperatures(self, faces, thermal_data):
        """Calculate temperature stats for each forehead region"""
        temp_stats = []
        for (x, y, w, h) in faces:
            # Define forehead region (upper middle part of the face)
            fx = x + int(w * 0.1)
            fw = int(w * 0.8)
            fy = y + int(h * 0.05)
            fh = int(h * 0.2)
            
            forehead_region = thermal_data[fy:fy+fh, fx:fx+fw]
            temp_stats.append({
                'max': np.max(forehead_region),
                'min': np.min(forehead_region),
                'mean': np.mean(forehead_region),
                'median': np.median(forehead_region),
                'face_roi': (x, y, w, h),
                'forehead_roi': (fx, fy, fw, fh),
                'face_roi_image': (scale_factor*x, scale_factor*y, scale_factor*w, scale_factor*h),
                'forehead_roi_image': (scale_factor*fx, scale_factor*fy, scale_factor*fw, scale_factor*fh)
            })
        return temp_stats
    

Only it is said that it should be useful to have real IR sensor  or termocamera but i can use an arbitrary mapping of this  value


In [16]:
temp = 34 + (intensity / 255.0) * 4 



<hr>    NEED TO SEE THE ESTIMATED TEMPERATURE <HR>


In [17]:
# Load the pre-trained face and smile cascade classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start the video capture (use webcam)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    gray = cv2.GaussianBlur(gray, (5, 5), 0.5)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 3,minSize=(100, 100))

    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Region of interest (ROI) for forehead detection
        x_forehead = x+int(w*0.1)
        w_forehead = int(w*0.8)
        y_forehead = y + int(h*0.05)
        h_forehead = int(h*0.2)
        forehead_region = gray[y_forehead:y_forehead + h_forehead, x_forehead:x_forehead + w_forehead] # Region around the forehead
        # Convert forehead ROI to grayscale and calculate average intensity
        intensity = np.mean(forehead_region)
        #temperature
        temp = 34 + (intensity / 255.0) * 4 
        #show intensity
        cv2.putText(frame, f"Temp: {int(temp)}", (x_forehead, y_forehead - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)


        # Draw rectangle around the forehead
        cv2.rectangle(frame, (x_forehead, y_forehead), (x_forehead+w_forehead, y_forehead + h_forehead), (0, 0, 255), 2)

        
    # Display the resulting frame
    cv2.imshow("Forehead temp", frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


SKELETON 

In [18]:
%pip install mediapipe --pre


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\utente\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [19]:
%pip install mediapipe

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: mediapipe in c:\users\utente\appdata\local\programs\python\python39\lib\site-packages (0.10.21)



You should consider upgrading via the 'c:\Users\utente\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [20]:
import cv2
import mediapipe as mp
import numpy as np

# Inizializza MediaPipe pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

# Avvia la webcam
cap = cv2.VideoCapture(0)

def get_forehead_roi(image, landmarks, w, h):
    # Usa punti tra occhi e naso per definire la fronte
    if landmarks[0].visibility < 0.5 or landmarks[1].visibility < 0.5 or landmarks[2].visibility < 0.5:
        return None

    # Landmark occhi e naso
    left_eye = landmarks[2]     # LEFT_EYE
    right_eye = landmarks[5]    # RIGHT_EYE
    nose = landmarks[0]         # NOSE

    # Coordinate pixel
    x = int((left_eye.x + right_eye.x) / 2 * w)
    y = int((nose.y - 0.1) * h)  # un po' sopra il naso = fronte

    roi_size = 30
    x1, y1 = max(0, x - roi_size), max(0, y - roi_size)
    x2, y2 = min(w, x + roi_size), min(h, y + roi_size)

    return x1, y1, x2, y2

while True:
    ret, frame = cap.read()
    if not ret:
        break

    h, w, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Ottieni bounding box della fronte
        roi_coords = get_forehead_roi(frame, results.pose_landmarks.landmark, w, h)
        if roi_coords:
            x1, y1, x2, y2 = roi_coords
            forehead_roi = frame[y1:y2, x1:x2]
            gray = cv2.cvtColor(forehead_roi, cv2.COLOR_BGR2GRAY)
            intensity = np.mean(gray)
            temperature = 34 + (intensity / 255.0) * 4

            # Visualizza ROI e temperatura
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{temperature:.2f} C", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Fronte non trovata", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow("Temperatura fronte (MediaPipe)", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


READING PATCH FILES 

In [21]:
import pandas as pd

# Percorsi dei file sul tuo PC (usa raw string oppure raddoppia i backslash)
file1 = r"C:\Users\utente\Desktop\MAGISTRALE\HEALTH DATA SCIENCE\n-health-lab\07 - temperature\1.csv"
file2 = r"C:\Users\utente\Desktop\MAGISTRALE\HEALTH DATA SCIENCE\n-health-lab\07 - temperature\2.csv"


# Leggi i file
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Filtra i dati validi
df1_valid = df1[df1["valid"] == True]
df2_valid = df2[df2["valid"] == True]

# Calcola le medie
media1 = df1_valid["temperatureProcessed"].mean()
media2 = df2_valid["temperatureProcessed"].mean()

# Stampa i risultati
print(f"Temperatura media file 1: {media1:.2f}°C")
print(f"Temperatura media file 2: {media2:.2f}°C")


Temperatura media file 1: 32.77°C
Temperatura media file 2: 32.77°C


DO IT WITH THREE VALUES  // n values 

In [22]:
import pandas as pd

# Percorsi dei file (modifica con i nomi reali dei file)
file1 = r"C:\Users\utente\Desktop\MAGISTRALE\HEALTH DATA SCIENCE\n-health-lab\07 - temperature\1.csv"
file2 = r"C:\Users\utente\Desktop\MAGISTRALE\HEALTH DATA SCIENCE\n-health-lab\07 - temperature\2.csv"
file3 = r"C:\Users\utente\Desktop\MAGISTRALE\HEALTH DATA SCIENCE\n-health-lab\07 - temperature\steadytemp_test.csv"

# Carica i dati
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

# Filtra solo le righe valide
df1_valid = df1[df1["valid"] == True]
df2_valid = df2[df2["valid"] == True]
df3_valid = df3[df3["valid"] == True]

# Calcola le medie individuali
media1 = df1_valid["temperatureProcessed"].mean()
media2 = df2_valid["temperatureProcessed"].mean()
media3 = df3_valid["temperatureProcessed"].mean()

# Calcola la media combinata pesata (sulla base del numero di osservazioni valide)
totale_valori = pd.concat([df1_valid["temperatureProcessed"],
                           df2_valid["temperatureProcessed"],
                           df3_valid["temperatureProcessed"]])
media_complessiva = totale_valori.mean()

# Stampa i risultati
print(f"Temperatura media file 1: {media1:.2f}°C")
print(f"Temperatura media file 2: {media2:.2f}°C")
print(f"Temperatura media file 3: {media3:.2f}°C")
print(f"Temperatura media complessiva: {media_complessiva:.2f}°C")


Temperatura media file 1: 32.77°C
Temperatura media file 2: 32.77°C
Temperatura media file 3: 36.29°C
Temperatura media complessiva: 35.60°C


live of the temperature steady 


In [23]:
%pip install bleak


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\utente\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


visualizzazione del sensore 

In [15]:
import asyncio
from bleak import BleakScanner

async def scan():
    print("🔍 Scanning BLE devices...")
    devices = await BleakScanner.discover()
    for d in devices:
        print(f"{d.name} - {d.address} - RSSI: {d.rssi}")

asyncio.run(scan())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [16]:
asyncio.run(scan())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [24]:
import asyncio
from bleak import BleakScanner

async def scan():
    print("🔍 Scanning BLE devices...")
    devices = await BleakScanner.discover()
    for d in devices:
        print(f"{d.name} - {d.address} - RSSI: {d.rssi}")

await scan() 

🔍 Scanning BLE devices...


BleakError: Thread is configured for Windows GUI but callbacks are not working.

In [31]:
import asyncio
from bleak import BleakScanner

async def scan():
    print("🔍 Scanning BLE devices per 10 secondi...")
    devices = await BleakScanner.discover(timeout=10.0)
    for d in devices:
        print(f"{d.name} - {d.address} - RSSI: {d.rssi}")

asyncio.run(scan())

RuntimeError: asyncio.run() cannot be called from a running event loop

finding the Mac with 3rd party 

POSSIBLE MACs
8C:79:F5:1C:7E:14


In [24]:
import asyncio
from bleak import BleakClient

ADDRESS = "8C:79:F5:1C:7E:14"  # Inserisci MAC del sensore


async def read_data(address, char_uuid):
    async with BleakClient(address) as client:
        value = await client.read_gatt_char(char_uuid)
        print("Temperatura:", int.from_bytes(value, byteorder='little') / 100)

asyncio.run(read_data(ADDRESS, CHAR_UUID))


NameError: name 'CHAR_UUID' is not defined